In [14]:
!pip install dash

   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ------------- -------------------------- 2.6/7.8 MB 15.1 MB/s eta 0:00:01
   -------------------------- ------------- 5.2/7.8 MB 13.3 MB/s eta 0:00:01
   ------------------------------------- -- 7.3/7.8 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 10.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install -U kaleido

   ---------------------------------------- 0.0/65.9 MB ? eta -:--:--
   - -------------------------------------- 2.4/65.9 MB 13.4 MB/s eta 0:00:05
   --- ------------------------------------ 5.0/65.9 MB 12.6 MB/s eta 0:00:05
   ---- ----------------------------------- 7.9/65.9 MB 12.8 MB/s eta 0:00:05
   ----- ---------------------------------- 9.4/65.9 MB 11.3 MB/s eta 0:00:06
   ------- -------------------------------- 12.1/65.9 MB 11.8 MB/s eta 0:00:05
   -------- ------------------------------- 14.2/65.9 MB 11.1 MB/s eta 0:00:05
   ---------- ----------------------------- 17.3/65.9 MB 11.6 MB/s eta 0:00:05
   ----------- ---------------------------- 19.7/65.9 MB 11.8 MB/s eta 0:00:04
   ------------- -------------------------- 22.8/65.9 MB 11.9 MB/s eta 0:00:04
   --------------- ------------------------ 25.7/65.9 MB 12.1 MB/s eta 0:00:04
   ----------------- ---------------------- 28.6/65.9 MB 12.2 MB/s eta 0:00:04
   ------------------- -------------------- 31.7/65.9 MB 12.4 MB/


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import dash
from dash import dcc, html, Input, Output, State
import networkx as nx
import pandas as pd
from PIL import Image
import base64
from io import BytesIO
import plotly.graph_objects as go
import kaleido

# Carregar o DataFrame
df = pd.read_csv("C:\\Users\\lumor\\Downloads\\resultados_queries\\resultados_gerais_trabalho3.csv")
available_queries = df["query"].unique()  # Queries disponíveis
available_years = df["year"].unique()  # Anos disponíveis

# Função para gerar o grafo e o layout Plotly
def create_graph(selected_queries):
    # Filtrar o DataFrame pelos queries e anos selecionados
    filtered_df = df[df["query"].isin(selected_queries)]

    # Criar o grafo
    G = nx.Graph()
    image_nodes = {}
    query_nodes = {}

    # Adicionar nodes e edges ao grafo
    for _, row in filtered_df.iterrows():
        image_id = row["image_id"]
        image_path = row["image_path"]
        query = row["query"]

        if image_id not in image_nodes:
            G.add_node(image_id, type="image")
            image_nodes[image_id] = image_path

        if query not in query_nodes:
            G.add_node(query, type="query")
            query_nodes[query] = True

        G.add_edge(image_id, query)

    # Gerar layout do grafo
    pos = nx.spring_layout(G, seed=42)

    # Criar a figura Plotly
    fig = go.Figure()

    # Adicionar edges ao grafo
    edge_x = []
    edge_y = []
    for source, target in G.edges():
        x1, y1 = pos[source]
        x2, y2 = pos[target]
        edge_x.extend([x1, x2, None])
        edge_y.extend([y1, y2, None])

    fig.add_trace(go.Scatter(
        x=edge_x,
        y=edge_y,
        mode="lines",
        line=dict(color="gray", width=1),
        hoverinfo="none"
    ))

    # Adicionar imagens como thumbnails nos nodes
    for node, image_path in image_nodes.items():
        try:
            img = Image.open(image_path)
            img.thumbnail((100, 100))  # Redimensionar para thumbnails

            # Converter imagem para base64
            buffered = BytesIO()
            img.save(buffered, format="PNG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode()

            x, y = pos[node]

            fig.add_layout_image(
                dict(
                    source=f"data:image/png;base64,{img_base64}",
                    x=x,
                    y=y,
                    xref="x",
                    yref="y",
                    sizex=0.15,  # Ajustar tamanho das thumbnails
                    sizey=0.15,
                    xanchor="center",
                    yanchor="middle",
                    layer="above"
                )
            )
        except Exception as e:
            print(f"Erro ao carregar a imagem {image_path}: {e}")

    # Adicionar queries como texto
    for query in query_nodes.keys():
        x, y = pos[query]
        fig.add_annotation(
            x=x,
            y=y + 0.1,  # Posicionar texto acima dos nodes
            text=query,
            showarrow=False,
            font=dict(size=14, color="blue"),
            xanchor="center",
            yanchor="middle"
        )

    # Layout final
    fig.update_layout(
        title="Grafo de Queries e Thumbnails",
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        showlegend=False,
        margin=dict(l=0, r=0, t=50, b=0),
        height=900,
    )

    return fig

# Criar a aplicação Dash
app = dash.Dash(__name__)

# Layout da aplicação
app.layout = html.Div([
    html.H1("Queries e Anos", style={"textAlign": "center"}),


    # Dropdown para seleção de queries
    html.Div([
        dcc.Dropdown(
            id="query-selector",
            options=[{"label": query, "value": query} for query in available_queries],
            multi=True,
            placeholder="Selecione os queries...",
            style={"width": "80%", "margin": "auto"}
        )
    ], style={"marginBottom": "20px"}),

    # Gráfico
    dcc.Graph(
        id="graph",
        config={"displayModeBar": False}  # Remover barra de ferramentas do Plotly
    ),

    # Botão para tirar screenshot
    html.Button("Take Screenshot", id="screenshot-btn", style={"marginTop": "10px"}),

    # Exibir o screenshot
    html.Div(id="screenshot-output", style={"marginTop": "20px"})
])

# Callback para atualizar o gráfico
@app.callback(
    Output("graph", "figure"),
    [Input("query-selector", "value")]
)
def update_graph(selected_queries, ):
    if not selected_queries:
        return go.Figure()  # Retorna gráfico vazio se nada for selecionado
    return create_graph(selected_queries)

# Callback para tirar screenshot do gráfico
@app.callback(
    Output("screenshot-output", "children"),
    [Input("screenshot-btn", "n_clicks")],
    [State("graph", "figure")],
    prevent_initial_call=True
)
def take_screenshot(n_clicks, figure):
    # Gerar imagem do gráfico
    fig = go.Figure(figure)
    
    # Caminho para salvar o arquivo
    file_path = "screenshot.png"
    
    # Salvar a imagem diretamente como PNG
    fig.write_image(file_path, format="png", width=1280, height=720)
    
    # Retornar mensagem de sucesso
    return html.Div([html.P(f"Screenshot saved as '{file_path}'. Check the current directory.")])

# Executar a aplicação
if __name__ == "__main__":
    app.run_server(debug=True)


Erro ao carregar a imagem E://The Delineator Cropped Images\Illustration/Photograph\1901_page_1618_extracted_5.png: [Errno 2] No such file or directory: 'E://The Delineator Cropped Images\\Illustration/Photograph\\1901_page_1618_extracted_5.png'
Erro ao carregar a imagem E://The Delineator Cropped Images\Illustration/Photograph\1912_page_915_extracted_1.png: [Errno 2] No such file or directory: 'E://The Delineator Cropped Images\\Illustration/Photograph\\1912_page_915_extracted_1.png'
Erro ao carregar a imagem E://The Delineator Cropped Images\Illustration/Photograph\1909_page_1021_extracted_1.png: [Errno 2] No such file or directory: 'E://The Delineator Cropped Images\\Illustration/Photograph\\1909_page_1021_extracted_1.png'
Erro ao carregar a imagem E://The Delineator Cropped Images\Illustration/Photograph\1914_page_624_extracted_2.png: [Errno 2] No such file or directory: 'E://The Delineator Cropped Images\\Illustration/Photograph\\1914_page_624_extracted_2.png'
Erro ao carregar a i